In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,40081
2,Huelva,Confirmados PDIA 14 días,3115
3,Huelva,Tasa PDIA 14 días,"607,011321784204"
4,Huelva,Confirmados PDIA 7 días,1378
5,Huelva,Tasa PDIA 7 dias,"268,5269988502835"
6,Huelva,Total Confirmados,40290
7,Huelva,Curados,34617
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  40081.0


/tmp/ipykernel_4195/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11172.0


/tmp/ipykernel_4195/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4195/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4195/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4195/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 407 personas en los últimos 7 días 

Un positivo PCR cada 170 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,40081.0,1378.0,3115.0,513170.0,268.526999,607.011322,290.0
Huelva-Costa,23541.0,779.0,1826.0,289548.0,269.040021,630.638098,165.0
Condado-Campiña,12574.0,468.0,920.0,156231.0,299.556426,588.871607,105.0
Huelva (capital),11172.0,353.0,848.0,143837.0,245.416687,589.556234,72.0
Ayamonte,1702.0,61.0,142.0,21104.0,289.044731,672.858226,23.0
Sierra de Huelva-Andévalo Central,3586.0,127.0,362.0,67391.0,188.452464,537.163716,20.0
Isla Cristina,2720.0,66.0,190.0,21393.0,308.512130,888.140981,17.0
Lepe,2624.0,85.0,197.0,27880.0,304.878049,706.599713,13.0
Moguer,1720.0,75.0,153.0,21867.0,342.982576,699.684456,13.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cumbres Mayores,113.0,21.0,52.0,1749.0,1200.686106,2973.127501,0.0
Encinasola,159.0,6.0,30.0,1310.0,458.015267,2290.076336,0.0
Corteconcepción,36.0,2.0,11.0,536.0,373.134328,2052.238806,1.0
Sanlúcar de Guadiana,14.0,7.0,7.0,403.0,1736.972705,1736.972705,2.0
Cañaveral de León,11.0,2.0,6.0,396.0,505.050505,1515.151515,0.0
Villablanca,236.0,19.0,35.0,2885.0,658.578856,1213.171577,4.0
Nava (La),5.0,2.0,3.0,258.0,775.193798,1162.790698,0.0
Beas,319.0,17.0,47.0,4341.0,391.614835,1082.699839,1.0
Trigueros,453.0,43.0,85.0,7862.0,546.934622,1081.149835,11.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Corteconcepción,36.0,2.0,11.0,536.0,373.134328,2052.238806,1.0,0.181818
Galaroza,49.0,1.0,5.0,1382.0,72.358900,361.794501,0.0,0.200000
Zalamea la Real,107.0,1.0,5.0,3054.0,32.743942,163.719712,0.0,0.200000
Cortegana,262.0,5.0,25.0,4602.0,108.648414,543.242069,1.0,0.200000
Encinasola,159.0,6.0,30.0,1310.0,458.015267,2290.076336,0.0,0.200000
Valverde del Camino,773.0,11.0,41.0,12750.0,86.274510,321.568627,2.0,0.268293
Nerva,236.0,6.0,22.0,5169.0,116.076611,425.614239,0.0,0.272727
Higuera de la Sierra,46.0,3.0,9.0,1291.0,232.378002,697.134005,2.0,0.333333
Cañaveral de León,11.0,2.0,6.0,396.0,505.050505,1515.151515,0.0,0.333333
